In [8]:
!pip install beautifulsoup4

In [9]:
import requests
from bs4 import BeautifulSoup
from datetime import date, timedelta
import csv

def get_game_ids():
    url = "https://www.frontierleague.com/schedule"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}
    session = requests.Session()
    response = session.get(url, headers=headers)
    game_ids = {}
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        box_score_buttons = soup.find_all('a', class_='link text-nowrap btn btn-outline-secondary btn-sm my-1')
        for box_score_button in box_score_buttons:
            try:
                game_id = box_score_button['href'].split('_')[1][:4]
                game_date = box_score_button['href'].split('/')[-1].split('_')[0]  # Extract date from the URL
                if game_date in game_ids:
                    game_ids[game_date].append(game_id)
                else:
                    game_ids[game_date] = [game_id]
            except (IndexError, KeyError):
                continue
    return game_ids


def get_home_plate_umpire(date, game_id):
    url = f"https://www.frontierleague.com/sports/bsb/2022-23/boxscores/{date}_{game_id}.xml"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}
    session = requests.Session()
    response = session.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        td_elements = soup.find_all('td', class_='text')
        for td in td_elements:
            if td.strong and td.strong.text.strip() == 'Umpires:':
                umpire_text = td.find_next_sibling('td').text.strip()
                umpires = [umpire.strip() for umpire in umpire_text.split(',')]
                home_plate_umpire = umpires[0].replace('Home plate -', '').strip()
                return home_plate_umpire
    return None

def get_game_teams(date, game_id):
    url = f"https://www.frontierleague.com/sports/bsb/2022-23/boxscores/{date}_{game_id}.xml"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}
    session = requests.Session()
    response = session.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        h1_element = soup.find('h1')
        try:
            teams_text = h1_element.text.strip()
        except AttributeError: 
            return None, None
        teams = teams_text.split(' at ')
        if len(teams) == 2:
            away_team = teams[0].split(' ', 3)[-1]
            home_team = teams[1]
            return home_team, away_team
    return None, None

start_date = date(2023, 5, 11)

today = date.today()

date_range = [start_date + timedelta(days=i) for i in range((today - start_date).days + 1)]

game_ids = get_game_ids()

with open('umpires.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Date', 'Game ID', 'Home Plate Umpire', 'Home Team', 'Away Team'])  # Write the header

    for current_date in date_range:
        formatted_date = current_date.strftime("%Y%m%d")

        game_ids_for_date = game_ids.get(formatted_date)
        if game_ids_for_date is None:
            continue
        for game_id in game_ids_for_date:
            home_plate_umpire = get_home_plate_umpire(formatted_date, game_id)
            home_team, away_team = get_game_teams(formatted_date, game_id)

            if home_plate_umpire:
                writer.writerow([formatted_date, game_id, home_plate_umpire, home_team, away_team])  # Write the data
                print(f"Date: {formatted_date}")
                print(f"Game ID: {game_id}")
                print(f"Home Plate Umpire: {home_plate_umpire}")
                print(f"Home Team: {home_team}")
                print(f"Away Team: {away_team}")
                print("---")
            else:
                print(f"No umpire information found for Game ID: {game_id}")
                print("---")

No umpire information found for Game ID: c6ff
---
Date: 20230511
Game ID: vpqx
Home Plate Umpire: Dave Speranza
Home Team:  New York Boulders
Away Team: New Jersey Jackals 
---
Date: 20230511
Game ID: efe0
Home Plate Umpire: Kevin Furman
Home Team:  Windy City ThunderBolts
Away Team: Schaumburg Boomers 
---
No umpire information found for Game ID: f72n
---
Date: 20230512
Game ID: e9jp
Home Plate Umpire: SCOTT COSTELLO
Home Team:  Ottawa Titans
Away Team: Empire State Greys 
---
Date: 20230512
Game ID: 54ks
Home Plate Umpire: Eric Carmona
Home Team:  New York Boulders
Away Team: New Jersey Jackals 
---
Date: 20230512
Game ID: nwls
Home Plate Umpire: Ty Unthank
Home Team:  Florence Y'alls
Away Team: Gateway Grizzlies 
---
Date: 20230512
Game ID: xswm
Home Plate Umpire: Paul Hess
Home Team:  Lake Erie Crushers
Away Team: Quebec Capitales 
---
Date: 20230512
Game ID: deq3
Home Plate Umpire: Kevin Furman
Home Team:  Evansville Otters
Away Team: Washington Wild Things 
---
Date: 20230512
Gam

No umpire information found for Game ID: nujc
---
Date: 20230521
Game ID: n3pr
Home Plate Umpire: Derek Moccio
Home Team:  Tri-City ValleyCats
Away Team: Washington Wild Things 
---
Date: 20230521
Game ID: t059
Home Plate Umpire: Jon Lamattina
Home Team:  New York Boulders
Away Team: Quebec Capitales 
---
Date: 20230523
Game ID: huyb
Home Plate Umpire: Mike Williams
Home Team:  New Jersey Jackals
Away Team: Evansville Otters 
---
Date: 20230523
Game ID: hcam
Home Plate Umpire: Brendan Boucher
Home Team:  Ottawa Titans
Away Team: Sussex County Miners 
---
Date: 20230523
Game ID: 7ztb
Home Plate Umpire: Mike O'Leary
Home Team:  Joliet Slammers
Away Team: New York Boulders 
---
Date: 20230523
Game ID: wiiw
Home Plate Umpire: Kevin Furman
Home Team:  Windy City ThunderBolts
Away Team: Tri-City ValleyCats 
---
Date: 20230523
Game ID: nbj6
Home Plate Umpire: Hugues Lafreniere
Home Team:  Trois-Rivieres Aigles
Away Team: Lake Erie Crushers 
---
Date: 20230523
Game ID: svbm
Home Plate Umpire: 

Date: 20230531
Game ID: p3zf
Home Plate Umpire: Reid Hoover
Home Team:  Evansville Otters
Away Team: Windy City ThunderBolts 
---
Date: 20230531
Game ID: umvq
Home Plate Umpire: Elijah Bester
Home Team:  Joliet Slammers
Away Team: New Jersey Jackals 
---
Date: 20230601
Game ID: 0k8u
Home Plate Umpire: Maxime Leroux
Home Team:  Ottawa Titans
Away Team: Schaumburg Boomers 
---
Date: 20230601
Game ID: hpv2
Home Plate Umpire: Jason Klump
Home Team:  Tri-City ValleyCats
Away Team: Trois-Rivieres Aigles 
---
No umpire information found for Game ID: 9b38
---
Date: 20230601
Game ID: uv0t
Home Plate Umpire: Todd Odell
Home Team:  Lake Erie Crushers
Away Team: Gateway Grizzlies 
---
Date: 20230601
Game ID: qgkb
Home Plate Umpire: Mike Williams
Home Team:  New York Boulders
Away Team: Empire State Greys 
---
Date: 20230601
Game ID: 6ttb
Home Plate Umpire: John Ramsey
Home Team:  Sussex County Miners
Away Team: Quebec Capitales 
---
Date: 20230601
Game ID: 4i8c
Home Plate Umpire: Preston Childers


Date: 20230610
Game ID: 0c8p
Home Plate Umpire: Jim Schaly
Home Team:  Lake Erie Crushers
Away Team: Windy City ThunderBolts 
---
Date: 20230610
Game ID: 5oej
Home Plate Umpire: Matt Van Houten
Home Team:  New Jersey Jackals
Away Team: Trois-Rivieres Aigles 
---
Date: 20230610
Game ID: j2sz
Home Plate Umpire: Steve Leonardo
Home Team:  Sussex County Miners
Away Team: New York Boulders 
---
Date: 20230610
Game ID: wikr
Home Plate Umpire: Clifton Glidwell
Home Team:  Florence Y'alls
Away Team: Empire State Greys 
---
Date: 20230610
Game ID: kzjw
Home Plate Umpire: Maxime Leroux
Home Team:  Quebec Capitales
Away Team: Washington Wild Things 
---
Date: 20230610
Game ID: rsc6
Home Plate Umpire: Drew Taylor
Home Team:  Evansville Otters
Away Team: Schaumburg Boomers 
---
Date: 20230610
Game ID: pv37
Home Plate Umpire: Joseph Brown
Home Team:  Gateway Grizzlies
Away Team: Tri-City ValleyCats 
---
Date: 20230611
Game ID: fsyy
Home Plate Umpire: Jared Merkley
Home Team:  Ottawa Titans
Away Team

Date: 20230620
Game ID: jrfe
Home Plate Umpire: Wesley Dunkle
Home Team:  Trois-Rivieres Aigles
Away Team: New York Boulders 
---
Date: 20230620
Game ID: futf
Home Plate Umpire: Hugues Lafreniere
Home Team:  Ottawa Titans
Away Team: Windy City ThunderBolts 
---
Date: 20230620
Game ID: 2w4p
Home Plate Umpire: Ben Levin
Home Team:  Florence Y'alls
Away Team: Gateway Grizzlies 
---
Date: 20230620
Game ID: fnj3
Home Plate Umpire: Jason Smith
Home Team:  Lake Erie Crushers
Away Team: Schaumburg Boomers 
---
Date: 20230620
Game ID: vng5
Home Plate Umpire: Yves Lamontagne
Home Team:  Quebec Capitales
Away Team: Tri-City ValleyCats 
---
Date: 20230620
Game ID: rp6j
Home Plate Umpire: Clark Morgan
Home Team:  Evansville Otters
Away Team: Sussex County Miners 
---
Date: 20230620
Game ID: jwy2
Home Plate Umpire: Jim Schaly
Home Team:  Joliet Slammers
Away Team: Washington Wild Things 
---
Date: 20230620
Game ID: 024v
Home Plate Umpire: Elijah Bester
Home Team:  Trois-Rivieres Aigles
Away Team: Ne

No umpire information found for Game ID: 5e5k
---
Date: 20230629
Game ID: xbdy
Home Plate Umpire: Joe Cruz
Home Team:  New York Boulders
Away Team: Washington Wild Things 
---
Date: 20230629
Game ID: j3fg
Home Plate Umpire: Marc Genest
Home Team:  Quebec Capitales
Away Team: Empire State Greys 
---
No umpire information found for Game ID: 0hiv
---
Date: 20230629
Game ID: m80s
Home Plate Umpire: Aaron Fink
Home Team:  Windy City ThunderBolts
Away Team: Joliet Slammers 
---
Date: 20230630
Game ID: s1gs
Home Plate Umpire: Scott Costello
Home Team:  Ottawa Titans
Away Team: Quebec Capitales 
---
Date: 20230630
Game ID: 3bib
Home Plate Umpire: Clark Morgan
Home Team:  New Jersey Jackals
Away Team: Tri-City ValleyCats 
---
Date: 20230630
Game ID: lbfa
Home Plate Umpire: Brian Miller
Home Team:  Lake Erie Crushers
Away Team: New York Boulders 
---
Date: 20230630
Game ID: h6sk
Home Plate Umpire: Tony Gisondi
Home Team:  Sussex County Miners
Away Team: Empire State Greys 
---
Date: 20230630
Gam

In [10]:
import pandas as pd

umps = pd.read_csv('umpires.csv')
main = pd.read_csv('master.csv')

In [11]:
umps['Home Plate Umpire'] = umps['Home Plate Umpire'].str.title()
umps['Date'] = pd.to_datetime(umps['Date'], format='%Y%m%d').dt.date
main['Date'] = pd.to_datetime(main['Date']).dt.strftime('%Y-%m-%d')
umps['Date'] = umps['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))


umps['Home Team'] = umps['Home Team'].str.strip()
umps['Away Team'] = umps['Away Team'].str.strip()

main['HomeTeam'] = main['HomeTeam'].replace(['TCVC', 'Tri city valleycats 2023', 'ValleyCats', 'Tri-City Valleycats'], 'Tri-City ValleyCats')
main['AwayTeam'] = main['AwayTeam'].replace(['TCVC', 'Tri city valleycats 2023', 'ValleyCats', 'Tri-City Valleycats'], 'Tri-City ValleyCats')

main['HomeTeam'] = main['HomeTeam'].replace(['Empire State Greys '], 'Empire State Greys')
main['AwayTeam'] = main['AwayTeam'].replace(['Empire State Greys '], 'Empire State Greys')

main['HomeTeam'] = main['HomeTeam'].replace(['Les Aigles De Trois Rivieres 2023', 'Trois-Rivieres Aigles', 'Trois Rivieres Aigles'], 'Trois Rivieres Aigles ')
main['AwayTeam'] = main['AwayTeam'].replace(['Les Aigles De Trois Rivieres 2023', 'Trois-Rivieres Aigles', 'Trois Rivieres Aigles'], 'Trois Rivieres Aigles ')

main['HomeTeam'] = main['HomeTeam'].replace(['Florence Y\'alls', 'Windy City ThunderBolts', 'Trois-Rivieres Aigles'], ['Florence Yalls', 'Windy City Thunderbolts', 'Trois Rivieres Aigles '])
main['AwayTeam'] = main['AwayTeam'].replace(['Florence Y\'alls', 'Windy City ThunderBolts', 'Trois-Rivieres Aigles'], ['Florence Yalls', 'Windy City Thunderbolts', 'Trois Rivieres Aigles '])


umps['Home Team'] = umps['Home Team'].replace(['Florence Y\'alls', 'Windy City ThunderBolts', 'Trois-Rivieres Aigles'], ['Florence Yalls', 'Windy City Thunderbolts', 'Trois Rivieres Aigles '])
umps['Away Team'] = umps['Away Team'].replace(['Florence Y\'alls', 'Windy City ThunderBolts', 'Trois-Rivieres Aigles'], ['Florence Yalls', 'Windy City Thunderbolts', 'Trois Rivieres Aigles '])

In [12]:
umps = umps.rename(columns={'Home Team': 'HomeTeam', 'Away Team': 'AwayTeam'})

In [13]:
umps

,Date,Game ID,Home Plate Umpire,HomeTeam,AwayTeam
0,2023-05-11,vpqx,Dave Speranza,New York Boulders,New Jersey Jackals
1,2023-05-11,efe0,Kevin Furman,Windy City Thunderbolts,Schaumburg Boomers
2,2023-05-12,e9jp,Scott Costello,Ottawa Titans,Empire State Greys
3,2023-05-12,54ks,Eric Carmona,New York Boulders,New Jersey Jackals
4,2023-05-12,nwls,Ty Unthank,Florence Yalls,Gateway Grizzlies
...,...,...,...,...,...
327,2023-07-01,m9fw,Silvio Martinez,Sussex County Miners,Empire State Greys
328,2023-07-01,hp40,Clifton Glidewell,Washington Wild Things,Schaumburg Boomers
329,2023-07-01,0qnu,Drew Taylor,Joliet Slammers,Evansville Otters
330,2023-07-01,visv,Chad Ozee,Windy City Thunderbolts,Florence Yalls


In [14]:
merged_df = pd.merge(main, umps, on=['Date', 'HomeTeam', 'AwayTeam'], how='inner')

In [15]:
df = merged_df.drop(['Unnamed: 0', 'Game ID'], axis=1).rename(columns={'HomeTeam_y': 'HomeTeam', 'AwayTeam_y': 'AwayTeam', 'Home Plate Umpire': 'Umpire'})

In [16]:
df.to_csv('masterWithUmps.csv')